In [ ]:
import pandas as pd
import numpy as np
from ProgSnap2 import ProgSnap2Dataset
from ProgSnap2 import PS2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import os
from os import path

## Loading and Cleaning Data
We load our data using the ProgSnap2Dataset class. This comes with both a main event table and a LinkTable giving students final exam data.

In [ ]:
semester = 'S19'
PATH = "data/CodeWorkout/" + semester

In [ ]:
data = ProgSnap2Dataset(PATH)

In [ ]:
# Drop the attempt column, since it's calculated incorrectly
data.drop_main_table_column('Attempt')

In [ ]:
main_table = data.get_main_table()
student_table = data.load_link_table('Subject')

In [ ]:
main_table.head()

### Fall 2019 Preprocessing

There were some differences between F19 and S19:
* In F19 there was an additional assignment (between Assignment 4 and 5), which only ~70% of students completed, likely additional optional practice. We will not use this assignment for prediction, since it is abnormal and not in S19. Since it comes in between the two assignments we are using for prediction, we simply remove it.
* In F19 the AssignmentIDs were renamed, so we will update their names
* In F19 2 ProblemIDs were renamed (though the solutions were unchanged), so we will update their names

In [ ]:
print(len(main_table[PS2.AssignmentID].unique()))
print(len(main_table[PS2.ProblemID].unique()))

In [ ]:
# This assignment has no analogue, but we use 500 to put it between the other 2
NEW_F19_ASSIGNMENT = 500

In [ ]:
if semester == 'F19':
    assignment_map = {
        597: 439,
        600: 487,
        609: 492,
        615: 494,
        622: NEW_F19_ASSIGNMENT,
        631: 502,
    }
    print(np.mean(main_table[PS2.AssignmentID].isin(assignment_map)))
    main_table[PS2.AssignmentID] = main_table[PS2.AssignmentID].map(assignment_map)
    
    # Two problems were renamed but are equivalent
    problem_map = {problem_id: problem_id for problem_id in main_table[PS2.ProblemID].unique()}
    problem_map[736] = 45
    problem_map[737] = 48
    print(np.mean(main_table[PS2.ProblemID].isin(problem_map)))
    main_table[PS2.ProblemID] = main_table[PS2.ProblemID].map(problem_map, na_action='ignore')
    
    # Overwrite the main table so this is the one that's copied
    data.set_main_table(main_table)

In [ ]:
print(len(main_table[PS2.AssignmentID].unique()))
print(len(main_table[PS2.ProblemID].unique()))

In [ ]:
main_table[PS2.ProblemID]

### Filtering Students

Here we remove studens who did not take the final exam, since we cannot use these for Task 2 (final exam score prediction). While this does somewhat bias the dataset for Task 1, it also ensures a consistent set of training/testing students for both tasks.

We can also see that few students are actually removed this way (381 -> 348 for S19).

In [ ]:
# Get the SubjectIDs where the final grade is non-0
# A 0 grade indicates the student did not take the final
print(len(student_table.index))
subject_ids = set(student_table[student_table['X-Grade'] != 0][PS2.SubjectID].unique())
subject_ids = subject_ids.intersection(set(student_table['SubjectID'].unique()))
len(subject_ids)

### EDA

We want to confirm that our selected students have a good and well-distributed number of attempts at all the problems in the dataset, and the most problems were well-attempted. The stats and figures below suggest that this is the case: most problems are attempted by ~300/350 students, and most students complete ~40/50 problems.

In [ ]:
# First we check how many problems each student attempted
main_table_filtered = main_table[main_table[PS2.SubjectID].isin(subject_ids)]
problems_per_student = main_table_filtered.groupby(by=['SubjectID']).apply(lambda rows: len(rows[PS2.ProblemID].unique()))
# 75% of problems were attempted by at least 40 studens, so that's good
problems_per_student.describe()

In [ ]:
# Only 3 attempted fewer than 10 problems
sum(problems_per_student < 10)

In [ ]:
# Next we see how many students attempted each problem
students_per_problem = main_table_filtered.groupby(by=['AssignmentID', 'ProblemID']).apply(lambda rows: len(rows[PS2.SubjectID].unique()))
students_per_problem.describe()

In [ ]:
# Next we plot the number of attemptes on each problem (x) / assignment (color)
from matplotlib.cm import viridis

assignment_ids = list(students_per_problem.keys().map(lambda x: x[0]))
assignment_ids = [sorted(assignment_ids).index(x) for x in assignment_ids]
colors = [viridis((float(i)-min(assignment_ids))/(max(assignment_ids)-min(assignment_ids))) for i in assignment_ids]

# There's a slight drop-off by assignment, but overall they're well-attempted
plt.bar(range(0, len(students_per_problem)), students_per_problem, color=colors)

## Creating the Class Label: Identifying Struggling Students

In any student modeling task, our goal is to predict if a student will struggle on the next problem. For this dataset, it's not obvious how to define that struggle.

We will define struggle as either:
1. Never getting a problem correct or 
2. Taking more attempts at a problem than 75% of students before getting it correct.

The code below justifies this decision by showing that most students get the problem correct _eventually_, and most student with more than the 75th percentil of attempts end up with many more attempts than their peers, indicating struggle.

### Aggregate data by problem

We first get all scored submissions (`Run.Program` events) and aggregate them by SubjectID and ProblemID, counting the number of attempts until a correct submission.

In [ ]:
runs = main_table_filtered[main_table_filtered[PS2.EventType] == 'Run.Program'].copy()
runs['TimeInt'] = pd.to_datetime(runs[PS2.ServerTimestamp]).apply(lambda x: x.value)

In [ ]:
def get_attempts(rows):
    scores = rows[PS2.Score]
    # If they scored 1, we return the first time they did so
    if (scores.max() == 1):
        # Argmax returns the first index of the highest score
        # Since the array is 0-indexed, we return +1
        return rows[PS2.Score].argmax() + 1
    return len(rows.index)
    

scores = runs.groupby([PS2.SubjectID, PS2.AssignmentID, PS2.ProblemID]).apply(get_attempts)
scores.describe()

In [ ]:
# Confirm that this is different than just the count of runs
runs.groupby([PS2.SubjectID, PS2.AssignmentID, PS2.ProblemID]).apply(lambda x: len(x.index)).describe()

In [ ]:
student_attempts = scores.to_frame('Attempts').reset_index()
print(student_attempts.shape)
student_attempts.head()

### Checking Eventual Success

Most student get each problem correct eventually, suggesting that the number of attempts is a more meaningful indicator of succeess.

In [ ]:
correct_eventually = runs.groupby([PS2.SubjectID, PS2.AssignmentID, PS2.ProblemID])[PS2.Score].apply(lambda x: max(x) == 1)

In [ ]:
np.mean(correct_eventually)

In [ ]:
correct_first_try = runs.groupby([PS2.SubjectID, PS2.AssignmentID, PS2.ProblemID])[PS2.Score].apply(lambda x: x.iloc[0] == 1)

In [ ]:
np.mean(correct_first_try)

In [ ]:
student_scores = student_attempts.merge(correct_eventually.to_frame('CorrectEventually'), on=[PS2.SubjectID, PS2.AssignmentID, PS2.ProblemID])
student_scores

In [ ]:
student_scores.CorrectEventually.value_counts()

### Choosing a cutoff for "struggling"
We choose the 75th percentile of attempts as the cutoff for struggling, and visualize this to verify that it meaningfully separates the "tail" of more struggling students from the main body. The chart below shows this for all 50 problems, and suggests that this is a reasonable (though by no means objectively correct) cutoff.

In [ ]:
problem_attempt_75th = student_scores.groupby(PS2.ProblemID).apply(lambda x: x.Attempts.quantile(0.75))
problem_attempt_75th.head()

In [ ]:
import matplotlib
matplotlib.rcParams['figure.figsize'] = [15, 10]

problem_ids = list(student_scores.ProblemID.unique())
fig, axs = plt.subplots(5, 10)
for i in range(5):
    for j in range(10):
        problem_id = problem_ids[i * 10 + j]
        attempts = student_scores[student_scores[PS2.ProblemID] == problem_id].Attempts
        p75 = problem_attempt_75th[problem_id] + 1
        axs[i, j].hist(attempts)
        axs[i, j].vlines(p75, 0, 50)

In [ ]:
cutoffs = student_scores['ProblemID'].apply(lambda x: problem_attempt_75th[x])
student_scores['Label'] = np.logical_and(student_scores['Attempts'] <= cutoffs, student_scores['CorrectEventually'])
student_scores['Label'].value_counts()

In [ ]:
matplotlib.rcParams['figure.figsize'] = [8, 4]

# The percentage of struggling problems per student is well-distributed
plt.hist(student_scores.groupby(PS2.SubjectID)['Label'].mean())

In [ ]:
np.mean(student_scores['Label'])

In [ ]:
len(student_scores.index)

## Identifying Late Assignments

Below we confirm that the 5 assignments are well-spaced out, with a consistent ordering accross students.

The latter 2 assignments are what is predicted in Task 1 of the data challenge.

We divide the data by assignment, rather than by problem, since within a given assignment studens do problems in a variety of orders (see analysis at the end of this document).

In [ ]:
problem_times = runs.groupby([PS2.AssignmentID, PS2.ProblemID])['TimeInt'].median()
start_time = min(problem_times)
problem_times = (problem_times - start_time) / 10**9

problem_successes = runs[runs[PS2.Score] == 1].groupby([PS2.AssignmentID, PS2.ProblemID])['TimeInt'].median()
problem_successes = (problem_successes - start_time) / 10**9

In [ ]:
problem_stats = problem_times.to_frame('MedTime').join(problem_successes.to_frame('MedSuccess')).reset_index()
problem_stats.head()

In [ ]:
y = range(0, len(problem_stats.index))
problem_stats.sort_values('MedTime', inplace=True)
plt.scatter(problem_stats['MedTime'], y, c='red')
# plt.scatter(problem_stats['MedSuccess'], y, c='blue')

In [ ]:
# Assignments are, thankfully, already in order
assignment_stats = runs.groupby(PS2.AssignmentID).apply(lambda x: np.median(x['TimeInt']))
assignment_stats

In [ ]:
valid_assignments = list(assignment_stats.sort_values().index)
if (semester == 'F19'):
    valid_assignments.remove(NEW_F19_ASSIGNMENT)
valid_assignments

In [ ]:
late_assignments = valid_assignments[-2:]
late_assignments

In [ ]:
early_assignments = valid_assignments[:-2]
early_assignments

In [ ]:
student_scores['IsLateProblem'] = student_scores[PS2.AssignmentID].isin(late_assignments)
student_scores.sort_values([PS2.SubjectID, PS2.AssignmentID, 'IsLateProblem', PS2.ProblemID], inplace=True)
# Remove attempts not in a valid assignment (for F19)
student_scores = student_scores[student_scores[PS2.AssignmentID].isin(valid_assignments)]
student_scores.head(50)

## Train/Test Split

Here we split out data into training/test datasets, as well as eary problems (used to extract features input into the model) and late problems (where struggle will be predicted).

### Split by SubjectID

In [ ]:
# Valid subjects must have completed at least one early and one late problem
ealry_late_subject_ids = student_scores.groupby(PS2.SubjectID)['IsLateProblem'].apply(lambda x: np.mean(x) > 0 and np.mean(x) < 1)
# The vast majority of students have
np.mean(ealry_late_subject_ids)

In [ ]:
# Instersect these subjectIDs with the ones who completed the final exam
valid_subject_ids = subject_ids.intersection(set(ealry_late_subject_ids.index[ealry_late_subject_ids]))
valid_subject_ids = list(valid_subject_ids)
valid_subject_ids.sort()

In [ ]:
print(len(subject_ids))
print(len(valid_subject_ids))

In [ ]:
train_ids, test_ids = train_test_split(list(valid_subject_ids), test_size=0.25, random_state=1234)

In [ ]:
# Check the train/test split dataframe against the last saved run
ids_df = pd.DataFrame({PS2.SubjectID: valid_subject_ids})
ids_df['IsTrain'] = ids_df[PS2.SubjectID].isin(train_ids)
ids_df

path = os.path.join('data', 'Release', semester)
os.makedirs(path, exist_ok=True)

# If saving, uncomment the top line
# ids_df.to_csv(os.path.join(path, 'split.csv'), index=False)
cached_df = pd.read_csv(os.path.join(path, 'split.csv'))

assert(ids_df.equals(cached_df))

In [ ]:
print(len(train_ids))
print(len(test_ids))

In [ ]:
student_scores_train = student_scores[student_scores[PS2.SubjectID].isin(train_ids)]
student_scores_test = student_scores[student_scores[PS2.SubjectID].isin(test_ids)]

In [ ]:
print(student_scores.shape[0])
print(student_scores_train.shape[0])
print(student_scores_test.shape[0])

### Task 1: Split Into early/late datasets
In Task 1, we need an early set of problems to use to extract features for the model, and a late set of problems where we're actually predicting student outcomes.

In [ ]:
early_train = student_scores_train[student_scores_train['IsLateProblem'] == False].drop(['IsLateProblem'], axis=1)
early_test = student_scores_test[student_scores_test['IsLateProblem'] == False].drop(['IsLateProblem'], axis=1)
late_train = student_scores_train[student_scores_train['IsLateProblem']].drop(['IsLateProblem', 'Attempts', 'CorrectEventually'], axis=1)
late_test = student_scores_test[student_scores_test['IsLateProblem']].drop(['IsLateProblem', 'Attempts', 'CorrectEventually'], axis=1)

In [ ]:
print(early_train.shape)
early_train.head()

In [ ]:
print(early_test.shape)
early_test.head()

In [ ]:
print(late_train.shape)
late_train.head()

In [ ]:
print(late_test.shape)
late_test.head()

In [ ]:
base_path = os.path.join('data', 'Release', semester)
os.makedirs(os.path.join(base_path, 'Train'), exist_ok=True)
os.makedirs(os.path.join(base_path, 'Test'), exist_ok=True)

In [ ]:
early_train.to_csv(os.path.join(base_path, 'Train', 'early.csv'), index=False)
late_train.to_csv(os.path.join(base_path, 'Train', 'late.csv'), index=False)
early_test.to_csv(os.path.join(base_path, 'Test', 'early.csv'), index=False)
late_test.drop('Label', axis=1).to_csv(os.path.join(base_path, 'Test', 'late.csv'), index=False)

In [ ]:
data.save_subset(os.path.join(base_path, 'Train', 'Data'), lambda df: df[df[PS2.SubjectID].isin(train_ids) & df[PS2.AssignmentID].isin(valid_assignments)])

In [ ]:
data.save_subset(os.path.join(base_path, 'Test', 'Data'), lambda df: df[df[PS2.SubjectID].isin(test_ids) & df[PS2.AssignmentID].isin(early_assignments)], False)

### Exporting the whole S19 Dataset

After the practice phase, we don't need the S19 test dataset anymore, so we can release the whole thing.

In [ ]:
if semester == 'S19':
    early_all = student_scores[student_scores['IsLateProblem'] == False].drop(['IsLateProblem'], axis=1)
    late_all = student_scores[student_scores['IsLateProblem']].drop(['IsLateProblem'], axis=1)

    base_path_all = os.path.join('data', 'Release', semester, 'All')
    os.makedirs(base_path_all, exist_ok=True)
    
    early_all.to_csv(os.path.join(base_path_all, 'early.csv'), index=False)
    late_all.to_csv(os.path.join(base_path_all, 'late.csv'), index=False)
    
    data.save_subset(os.path.join(base_path_all, 'Data'), lambda df: df[df[PS2.AssignmentID].isin(valid_assignments)])


### Task 2: Predicting Student Grades
For Task 2, we are predicting students' Final Exam grades. We just add the appropriate LinkTable _without_ the actual grades, just leaving the SubjectIDs to predict for.

In [ ]:
student_table_dir = os.path.join(base_path, 'Test', 'Data', 'LinkTables')
os.makedirs(student_table_dir, exist_ok=True)
student_table_test = student_table[student_table[PS2.SubjectID].isin(test_ids)]
student_table_test.drop('X-Grade', axis=1).to_csv(os.path.join(student_table_dir, 'Subject.csv'), index=False)

### Save the solution
The solution is, of course, not released, but used to judge submissions.

In [ ]:
sol_path = os.path.join('data', 'Solution', semester, 'task1', 'ref')
os.makedirs(sol_path, exist_ok=True)
late_test.to_csv(os.path.join(sol_path, 'truth.csv'), index=False)

In [ ]:
sol_path = os.path.join('data', 'Solution', semester, 'task2', 'ref')
os.makedirs(sol_path, exist_ok=True)
student_table_test.to_csv(os.path.join(sol_path, 'truth.csv'), index=False)

## Unused code for identifying the late problems

The code below was used to investigate whether we could predict the last 3 problems of every assignment using the first 7. However, students appear to do the assignment in a variety of orders, making this difficult. Additionally, this would leak future data (e.g. the first 7 problems on Assignment 2 could be used to predict the last 3 problems on Assignment 1).

In [ ]:
def get_start_end_order(rows):
    return pd.Series({
        'StartEventOrder': min(rows[PS2.Order])
        # 'EndEventOrder': max(rows[PS2.Order])
    })

start_orders = main_table_filtered.groupby([PS2.SubjectID, PS2.AssignmentID, PS2.ProblemID]).apply(get_start_end_order)
start_orders

In [ ]:
# Unfortunately, the last 3 problems aren't always easy to pick apart
print(problem_stats[PS2.AssignmentID].unique())
assignment1 = problem_stats[problem_stats[PS2.AssignmentID] == 439]
y = range(0, 10)
plt.scatter(assignment1['MedTime'], y, c='red')
plt.scatter(assignment1['MedSuccess'], y, c='blue')

In [ ]:
late_problems = problem_stats.groupby(PS2.AssignmentID).apply(lambda rows: list(rows[PS2.ProblemID][rows['MedTime'].argsort()][-3:]))
late_problems

In [ ]:
late_problem_ids_old = [st for row in late_problems for st in row]
late_problem_ids_old

In [ ]:
runs_time = runs.copy()
runs_time['IsLateProblemInAssignment'] = runs_time[PS2.ProblemID].isin(late_problem_ids_old)
runs_time.head(15)

In [ ]:
first_late_attempts = runs_time[runs_time['IsLateProblemInAssignment']].groupby([PS2.AssignmentID, PS2.SubjectID])['TimeInt'].min()
first_late_attempts

In [ ]:
runs_late = runs_time.merge(first_late_attempts.to_frame('FirstLateAttempt'), on=[PS2.AssignmentID, PS2.SubjectID], how='left')
np.mean(runs_late['FirstLateAttempt'].isna())
# ~2% of student-assignments did not have a late attempt

In [ ]:
runs_late['IsLateAttempt'] = runs_late['TimeInt'] >= runs_late['FirstLateAttempt']

In [ ]:
np.sum(~runs_late['IsLateAttempt'] & runs_late['IsLateProblemInAssignment'])

In [ ]:
# 30% of attempts at early problems occurred after the first attempt at a late problem
np.sum(runs_late['IsLateAttempt'] & ~runs_late['IsLateProblemInAssignment']) / np.sum(~runs_late['IsLateProblemInAssignment'])

In [ ]:
np.random.seed(1234)
subjects_sample = np.random.choice(runs_time[PS2.SubjectID].unique(), 20)
assignment_sample = runs_time[(runs_time[PS2.AssignmentID] == 439) & runs_time[PS2.SubjectID].isin(subjects_sample)]
assignment_sample.shape

In [ ]:
all_subject_ids = assignment_sample[PS2.SubjectID]
distinct_subject_ids = all_subject_ids.unique()
subject_indices = [sorted(distinct_subject_ids).index(x) for x in all_subject_ids]
colors = [viridis(float(i)) for i in assignment_sample['IsLateProblemInAssignment']]
subject_times_norm = assignment_sample.groupby('SubjectID')['TimeInt'].transform(lambda x: (x - x.mean()) / x.std())
widths = list(assignment_sample[PS2.Score].apply(lambda x: 0.2 if x < 1 else 3))
plt.scatter(x=subject_times_norm, y=subject_indices, color=colors, linewidths=widths, edgecolors=None)
plt.xlim([-2, 2])

In [ ]:
is_sorted = lambda a: np.all(a[:-1] <= a[1:])

def is_consistent(rows):
    orders = rows[PS2.Order]
    times = rows['TimeInt']
    return is_sorted(times.values)

consistent = runs.groupby([PS2.SubjectID, PS2.AssignmentID, PS2.ProblemID]).apply(is_consistent)
consistent

In [ ]:
runs[(runs[PS2.ProblemID]==102) & (runs[PS2.SubjectID]=='71ffa17407d66e134442eebb32d330ec')]

In [ ]:
consistent[~consistent]